### 1. Calculate byte-entropy of a given binary file:

Byte entropy is a measure of the average amount of information contained in each byte of a file. It helps us understand how compressible the file is - lower entropy generally means more compressible data.

The process involves these steps:
1. Read the binary file
2. Count occurrences of each byte (0-255)
3. Calculate probabilities (pi = Ni / N)
4. Calculate entropy using the formula H(p) = -Σ(pi log2(pi))

#### Theory:
- Entropy, in information theory, quantifies the average amount of information in a message.
- For a set of events with probabilities p1, p2, ..., pn, the entropy H is defined as:
  H = -Σ(pi * log2(pi))
- In our case, each "event" is the occurrence of a specific byte value (0-255).
- The unit of entropy is bits when using log2.

#### Implementation details:

1. **File Reading**: 
   We open the file in binary mode ('rb') to read raw bytes without any encoding.

2. **Byte Counting**:
   We use a list comprehension with the `count()` method to efficiently count occurrences of each byte value (0-255).

3. **Probability Calculation**:
   We divide each byte count by the total number of bytes to get probabilities.

4. **Entropy Calculation**:
   - We use a list comprehension with a conditional statement to handle the case where p = 0 (as log(0) is undefined).
   - The `math.log2()` function is used for logarithm calculation.
   - We use the `sum()` function to efficiently sum up all the terms.

5. **Output**:
   We return both the calculated entropy and the file content for further use.

In [ ]:
import math  # Import the math module for logarithm calculations

def calculate_byte_entropy(file_path):
    # Open the file in binary read mode
    with open(file_path, 'rb') as file:
        content = file.read()  # Read the entire file content
    
    N = len(content)  # Get the total number of bytes in the file
    
    # Count occurrences of each byte (0-255) in the file
    byte_counts = [content.count(i) for i in range(256)]
    
    # Calculate probability for each byte
    probabilities = [count / N for count in byte_counts]
    
    # Calculate entropy using the formula: H(p) = -Σ(pi * log2(pi))
    # We use a list comprehension and sum() for efficiency
    # The condition 'if p != 0 else 0' is to handle cases where p = 0 (log(0) is undefined)
    entropy = -sum(p * math.log2(p) if p != 0 else 0 for p in probabilities)
    
    return entropy, content  # Return both the calculated entropy and the file content

# Test the function
file_path = 'input_shannon_fano_huffman.bin'  # Path to the input file
entropy, content = calculate_byte_entropy(file_path)  # Call the function
print(f"Byte entropy: {entropy:.2f}")  # Print the calculated entropy, rounded to 2 decimal places

# Calculate byte counts
byte_counts = [content.count(i) for i in range(256)]

# Calculate probabilities
N = len(content)
probabilities = [count / N for count in byte_counts]

# Print byte occurrences and probabilities
for byte, (count, prob) in enumerate(zip(byte_counts, probabilities)):
    if count > 0:  # Only print for bytes that actually appear in the file
        char = chr(byte)  # Convert byte to its character representation
        print(f"{char}: Appears {count} times, Probability: {prob:.4f}")

##### Analysing test results:
- This is the calculated entropy of the file, measured in bits per byte.
- A value of 2.20 suggests that the file has some redundancy and is potentially compressible.
- The maximum entropy for a byte (8 bits) would be 8, which would occur if all 256 possible byte values were equally likely.
- Our result of 2.20 indicates that the file's content is not uniformly distributed, which is good for compression.

### 2. Create Shannon-Fano and Huffman code:
#### 2.1 Shannon-Fano:

Shannon-Fano coding is a technique for constructing a prefix code based on a set of symbols and their probabilities. It's a foundational algorithm in information theory and data compression.

The process involves these steps:
1) Create a Symbol class to represent each unique byte and its probability
2) Sort the symbols by probability in descending order
3) Implement the Shannon-Fano algorithm to assign codes
4) Encode the input file using the generated codes
5) Write the encoded data to a file

#### Theory:
- Shannon-Fano coding aims to create shorter codes for more frequent symbols and longer codes for less frequent ones.
- It's a "top-down" approach, recursively dividing the set of symbols into two subsets.
- The algorithm ensures that no codeword is a prefix of another, allowing for unambiguous decoding.
- While not optimal like Huffman coding, it often provides good compression ratios.

#### Implementation details:

1. **Symbol Class**:
   - We define a `Symbol` class to hold each byte value, its probability, and its assigned code.
   - This object-oriented approach makes it easier to manage and manipulate symbol data.

2. **Shannon-Fano Algorithm**:
   - The `shannon_fano_coding` function implements the core algorithm:
     - It recursively divides the list of symbols into two groups.
     - The split point is chosen to minimize the difference in total probability between the groups.
     - '0' is assigned to symbols in the first group, '1' to the second.
     - This process continues until each group contains only one symbol.

3. **Encoding Process**:
   - The `encode_file` function applies the generated codes to compress the input file:
     - It first writes the symbol table to the output file for later decoding.
     - It then reads the input file byte by byte, replacing each byte with its code.
     - The encoded data is written to the output file in 8-bit chunks for efficiency.

4. **Decoding Process**:
   - The `decode_file` function reverses the encoding:
     - It reads the symbol table from the encoded file.
     - It then reads the encoded data bit by bit, translating code sequences back to original bytes.

5. **File Handling**:
   - We use context managers (`with` statements) for safe and efficient file operations.
   - The encoded file structure includes:
     a) The symbol table for decoding
     b) The encoded data
     c) The total number of encoded bits (to handle padding in the last byte)

In [ ]:
import os    # For file and path operations
import filecmp  # For comparing files

# Class to represent a symbol (byte) with its probability and code
class Symbol:
    def __init__(self, value, probability):
        self.value = value          # The byte value (0-255)
        self.probability = probability  # Probability of occurrence
        self.code = ""              # Will store the assigned code

# Recursive function to implement Shannon-Fano coding algorithm
def shannon_fano_coding(symbols):
    if len(symbols) == 1:
        return  # Base case: only one symbol, no need to split

    # Calculate total probability of all symbols in the current group
    total_probability = sum(symbol.probability for symbol in symbols)
    current_sum = 0
    differences = []

    # Find the split point that minimizes the difference between the two groups
    for i, symbol in enumerate(symbols):
        current_sum += symbol.probability
        remainder = total_probability - current_sum
        differences.append(abs(current_sum - remainder))

    # Index where the difference is minimum is our split point
    split_index = differences.index(min(differences))

    # Assign '0' to the first group and '1' to the second group
    for i, symbol in enumerate(symbols):
        if i <= split_index:
            symbol.code += "0"
        else:
            symbol.code += "1"

    # Recursively apply the algorithm to each group
    shannon_fano_coding(symbols[:split_index + 1])
    shannon_fano_coding(symbols[split_index + 1:])

# Function to encode the file using the generated codes
def encode_file(input_file, output_file, symbols):
    # Create a dictionary for quick lookup of codes
    symbol_dict = {symbol.value: symbol.code for symbol in symbols}
    
    with open(input_file, 'rb') as infile, open(output_file, 'wb') as outfile:
        # Write the symbol table to the output file
        for symbol in symbols:
            outfile.write(f"{symbol.value}:{symbol.code},".encode('utf-8'))
        outfile.write(b'\n')  # End of symbol table marker

        encoded = ''  # String to hold bits before writing to file
        total_bits = 0  # Counter for total number of bits encoded
        while True:
            byte = infile.read(1)  # Read one byte from input file
            if not byte:
                break  # End of file
            encoded += symbol_dict[byte[0]]  # Append the code for this byte
            total_bits += len(symbol_dict[byte[0]])  # Increment total bits
            
            # Write complete bytes (8 bits) to the output file
            while len(encoded) >= 8:
                outfile.write(bytes([int(encoded[:8], 2)]))
                encoded = encoded[8:]
        
        # Write any remaining bits, padding with zeros if necessary
        if encoded:
            outfile.write(bytes([int(encoded.ljust(8, '0'), 2)]))
        
        # Write the total number of bits (4 bytes, big endian)
        outfile.write(total_bits.to_bytes(4, byteorder='big'))

# Function to decode the file using the generated codes
def decode_file(input_file, output_file, symbols):
    # Create a reverse lookup dictionary
    code_dict = {symbol.code: symbol.value for symbol in symbols}
    
    with open(input_file, 'rb') as infile, open(output_file, 'wb') as outfile:
        # Read the symbol table
        symbol_table = infile.readline().decode('utf-8').strip().split(',')
        symbols = [Symbol(int(s.split(':')[0]), 0) for s in symbol_table if s]
        for s in symbols:
            s.code = symbol_table[[int(x.split(':')[0]) for x in symbol_table if x].index(s.value)].split(':')[1]
        
        # Read the encoded data
        encoded_data = infile.read()
        
        # The last 4 bytes contain the total number of bits
        total_bits = int.from_bytes(encoded_data[-4:], byteorder='big')
        encoded_data = encoded_data[:-4]  # Remove the last 4 bytes
        
        # Convert to binary string
        binary_string = ''.join(format(byte, '08b') for byte in encoded_data)
        binary_string = binary_string[:total_bits]  # Trim to the correct number of bits
        
        # Decode
        current_code = ''
        for bit in binary_string:
            current_code += bit
            if current_code in code_dict:
                outfile.write(bytes([code_dict[current_code]]))
                current_code = ''

# Main execution
input_file = 'input_shannon_fano_huffman.bin'  # Path to input file
output_file = 'encoded_shannon_fano.bin'  # Path to output encoded file
decoded_file = 'decoded_shannon_fano.bin'  # Path to output decoded file

# Read the entire content of the input file
with open(input_file, 'rb') as f:
    content = f.read()

# Count occurrences of each byte value
byte_counts = [content.count(i) for i in range(256)]
total_bytes = len(content)  # Total number of bytes in the file

# Create Symbol objects for each byte that appears in the file
symbols = [Symbol(i, count/total_bytes) for i, count in enumerate(byte_counts) if count > 0]

# Sort symbols by probability in descending order
symbols.sort(key=lambda x: x.probability, reverse=True)

# Apply Shannon-Fano coding to generate codes
shannon_fano_coding(symbols)

# Print the generated codes
print("Shannon-Fano Codes:")
for symbol in symbols:
    print(f"Symbol {symbol.value}: {symbol.code}")

# Encode the file using the generated codes
encode_file(input_file, output_file, symbols)

# Decode the file
decode_file(output_file, decoded_file, symbols)

# Print file size comparison
print(f"Original file size: {os.path.getsize(input_file)} bytes")
print(f"Compressed file size: {os.path.getsize(output_file)} bytes")
print(f"Decompressed file size: {os.path.getsize(decoded_file)} bytes")

# Compare the original and decoded files
if filecmp.cmp(input_file, decoded_file):
    print("\nThe files have the same content.")
else:
    print("\nThe files do not have the same content.")

#### 2.2 Huffman:

Huffman coding is an optimal prefix coding algorithm used for lossless data compression. It creates variable-length codes for symbols, with shorter codes assigned to more frequent symbols.

The process involves these steps:
1. Create leaf nodes for each symbol.
2. Build the Huffman tree:
    - Select two nodes with the lowest frequency.
    - Create a new internal node with these two as children.
    - Assign the sum of the two frequencies to this new node.
    - Repeat until only one node remains (the root).
3. Generate Huffman codes by traversing the tree.
4. Encode the input file using the generated codes.
5. Implement decoding process.

#### Theory:
- Huffman coding creates an optimal prefix code based on the frequency of symbols.
- It guarantees that no code is a prefix of another, ensuring unambiguous decoding.
- The algorithm builds a binary tree (Huffman tree) bottom-up, unlike Shannon-Fano's top-down approach.
- Huffman coding is proven to produce an optimal code for a given set of frequencies and number of symbols.

#### Implementation details:

1. **Node Class**:
   - We define a `Node` class to represent nodes in the Huffman tree.
   - Each node contains a symbol, probability, left and right children, and a code.

2. **Building Huffman Tree**:
   - The `build_huffman_tree` function constructs the Huffman tree:
     - It starts with leaf nodes for each symbol.
     - It repeatedly selects the two nodes with lowest probabilities, creates a new parent node, and adds it back to the list.
     - This process continues until only one node (the root) remains.

3. **Generating Codes**:
   - The `print_nodes` function traverses the tree to generate and print Huffman codes.
   - It uses recursive depth-first traversal, appending '0' for left branches and '1' for right branches.

4. **Encoding Process**:
   - The `encode_string` function uses the Huffman tree to encode the input data.
   - It replaces each symbol with its corresponding Huffman code.

5. **File Handling**:
   - We use functions to save the Huffman tree structure to a string and read it back.
   - This allows us to store the tree structure in the compressed file for later decoding.

6. **Decoding Process**:
   - The `decode_huffman` function reads the encoded bit string and translates it back to the original symbols using the Huffman tree.

7. **Compression and Decompression**:
   - We implement the full process of reading an input file, compressing it, writing to a file, reading the compressed file, and decompressing it.
   - We also compare the original and decompressed files to verify correctness.

In [ ]:
import os    # For file and path operations
import filecmp  # For comparing files

# Class to represent a node in the Huffman tree
class Node:
    def __init__(self, probability, symbol, left=None, right=None):
        self.probability = probability  # Probability of the symbol
        self.symbol = symbol            # The symbol (byte value)
        self.left = left                # Left child node
        self.right = right              # Right child node
        self.code = ''                  # Huffman code for this node

# Function to build the Huffman tree
def build_huffman_tree(characters, probabilities):
    # Create initial nodes for each character
    nodes = [Node(probabilities[i], characters[i]) for i in range(len(characters))]
    
    while len(nodes) > 1:
        # Sort nodes by probability (ascending)
        nodes.sort(key=lambda x: x.probability)
        # Take the two nodes with lowest probabilities
        left = nodes.pop(0)
        right = nodes.pop(0)
        # Assign '0' to left edge and '1' to right edge
        left.code = 0
        right.code = 1
        # Create a new internal node with these two as children
        new_node = Node(left.probability + right.probability, left.symbol + right.symbol, left, right)
        # Add the new node back to the list
        nodes.append(new_node)
    
    # Return the root of the Huffman tree
    return nodes[0]

# Function to print the Huffman codes for each symbol
def print_nodes(node, value=''):
    new_value = value + str(node.code)
    if node.left:
        print_nodes(node.left, new_value)
    if node.right:
        print_nodes(node.right, new_value)
    if not node.left and not node.right:
        print(f"{node.symbol} -> {new_value}")

# Function to encode a string using the Huffman tree
def encode_string(string, root):
    encoded_string = ""
    for character in string:
        encoded_string += find_code(character, root)
    return encoded_string

# Function to find the Huffman code for a given symbol
def find_code(symbol, node, current_code=""):
    if not node:
        return None
    if node.symbol == symbol:
        return current_code
    else:
        left_code = find_code(symbol, node.left, current_code + "0")
        right_code = find_code(symbol, node.right, current_code + "1")
        if left_code:
            return left_code
        elif right_code:
            return right_code

# Function to save the Huffman tree structure as a string
def save_nodes_to_string(node, value=''):
    new_value = value + str(node.code)
    node_information = ""
    if node.left:
        node_information += save_nodes_to_string(node.left, new_value)
    if node.right:
        node_information += save_nodes_to_string(node.right, new_value)
    if not node.left and not node.right:
        node_information += f"{node.symbol}:{new_value} "
    return node_information

# Class to represent a symbol and its code when reading from file
class LoadedSymbol:
    def __init__(self, value, code):
        self.value = value  # The symbol (byte value)
        self.code = code    # The Huffman code for this symbol

# Function to read the Huffman tree structure from the encoded file
def read_first_line_from_binary_file_huffman(path):
    with open(path, 'rb') as file:
        first_line_bin = file.readline().decode('utf-8').strip()
        symbols_raw = first_line_bin.split()
        symbols = []
        for symbol_raw in symbols_raw:
            value, code = symbol_raw.split(":")
            symbol = LoadedSymbol(value, code)
            symbols.append(symbol)
    return symbols

# Function to read the encoded data from the file
def read_rest_from_binary_file_huffman(path):
    with open(path, 'rb') as file:
        file.readline()  # Skip the first line (symbol table)
        data = file.read()
    
    # The last 4 bytes contain the total number of bits
    total_bits = int.from_bytes(data[-4:], byteorder='big')
    data = data[:-4]  # Remove the last 4 bytes
    
    # Convert to binary string
    bit_string = ''.join(format(byte, '08b') for byte in data)
    return bit_string[:total_bits]  # Return only the valid bits

# Function to decode the Huffman-encoded data
def decode_huffman(bit_string, symbols):
    decoded_string = bytearray()
    code_dict = {symbol.code: int(symbol.value) for symbol in symbols}
    current_code = ""
    for bit in bit_string:
        current_code += bit
        if current_code in code_dict:
            decoded_string.append(code_dict[current_code])
            current_code = ""
    return decoded_string

# Main execution
input_file = 'input_shannon_fano_huffman.bin'  # Input file path
output_file = 'encoded_huffman.bin'  # Encoded file path
decoded_file = 'decoded_huffman.bin'  # Decoded file path

# Read the entire content of the input file
with open(input_file, 'rb') as f:
    content = f.read()

# Count occurrences of each byte value
byte_counts = [content.count(i) for i in range(256)]
total_bytes = len(content)

# Create characters and probabilities lists
characters = [i for i, count in enumerate(byte_counts) if count > 0]
probabilities = [count/total_bytes for count in byte_counts if count > 0]

# Build the Huffman tree and get codewords
huffman_tree_root = build_huffman_tree(characters, probabilities)
print("ENCODING:\n")
print("Huffman codewords:")
print_nodes(huffman_tree_root)

# Encode the input alphabet
encoded_string = encode_string(content, huffman_tree_root)
huffman_length = len(encoded_string)

# Convert the encoded string to bytes
bytes_array = bytearray()
for i in range(0, len(encoded_string), 8):
    byte = int(encoded_string[i:i+8].ljust(8, '0'), 2)
    bytes_array.append(byte)

# Write the encoded data to file
with open(output_file, "wb") as file:
    node_information = save_nodes_to_string(huffman_tree_root)
    file.write(node_information.encode("utf-8"))
    file.write(b"\n")
    file.write(bytes_array)
    file.write(huffman_length.to_bytes(4, byteorder='big'))

# Read the Huffman tree structure and encoded data from file
symbols_from_binary_file = read_first_line_from_binary_file_huffman(output_file)
content_to_decode = read_rest_from_binary_file_huffman(output_file)

print("\nDECODING:\n")
for symbol in symbols_from_binary_file:
    print(f"Symbol: {symbol.value}: {symbol.code}")

# Decode the data
decoded_string = decode_huffman(content_to_decode, symbols_from_binary_file)

# Write the decoded data to file
with open(decoded_file, 'wb') as file:
    file.write(decoded_string)

# Compare the original and decoded files
if filecmp.cmp(input_file, decoded_file):
    print("\nFiles have the same content.")
else:
    print("\nFiles do not have the same content.")

# Print file size comparison
print(f"Original file size: {os.path.getsize(input_file)} bytes")
print(f"Compressed file size: {os.path.getsize(output_file)} bytes")
print(f"Decompressed file size: {os.path.getsize(decoded_file)} bytes")

### 3.1 LZ77:

LZ77 is a dictionary-based lossless compression algorithm that uses a sliding window technique. It's part of the LZ (Lempel-Ziv) family of algorithms and forms the basis for many popular compression methods.

#### Theory:
- LZ77 works by replacing repeated occurrences of data with references to a single copy of that data existing earlier in the input stream.
- It maintains a "sliding window" divided into two parts:
  1. A search buffer (dictionary) containing recently processed data.
  2. A look-ahead buffer containing the next portion of data to be encoded.
- The algorithm looks for matches between the look-ahead buffer and the search buffer.
- It encodes data as triplets: (offset, length, next symbol)
  - offset: distance to the start of the match in the search buffer
  - length: length of the match
  - next symbol: the next symbol after the match

#### Implementation details:

1. **Compression Function (`lz77_compression`)**:
   - Input: The string to be compressed and the window size.
   - Output: A list of tuples representing the compressed data.
   - Process:
     - Iterate through the input string.
     - For each position, search for the longest match in the sliding window.
     - If a match is found, output (1, offset, length).
     - If no match is found, output (0, current_character).

2. **Writing to Binary File (`lz77_write_to_binary_file`)**:
   - This function writes the compressed data to a binary file.
   - It uses a simple encoding scheme:
     - '0' byte for unmatched characters
     - '1' byte for matches, followed by 2 bytes each for offset and length

3. **Reading from Binary File (`lz77_read_from_binary_file`)**:
   - This function reads the compressed data from the binary file.
   - It reverses the encoding process used in writing.

4. **Decompression Function (`lz77_decompression`)**:
   - Input: The compressed data (list of tuples).
   - Output: The decompressed string.
   - Process:
     - Iterate through the compressed data.
     - For unmatched characters (0, char), append the character.
     - For matches (1, offset, length), copy the specified sequence from the already decompressed data.

5. **File Handling and Verification**:
   - The implementation includes functions to read input files, write compressed data, and verify the decompression by comparing with the original file.

#### Key Points:
- The window size is a crucial parameter that affects both compression ratio and performance.
- Larger window sizes can potentially find more matches but increase search time.
- The simple (0/1) marker system in the binary file is efficient but limits the maximum offset and length to 65535 (2^16 - 1).
- This implementation balances simplicity and effectiveness, making it easy to understand while still providing good compression for many types of data.

In [ ]:
import os
import filecmp

# Input file path
input_file = 'input_lz77.bin'

# Read the entire content of the input file
with open(input_file, 'rb') as file:
    input_alphabet = file.read()

def lz77_compression(input_string, window_size):
    """
    Compress the input string using LZ77 algorithm.
    
    :param input_string: The string to be compressed
    :param window_size: The size of the sliding window
    :return: A list of tuples representing the compressed data
    """
    compressed_output = []
    i = 0  # Current position in the input string

    while i < len(input_string):
        max_match_length = 0
        max_match_index = 0

        # Search for the longest match in the sliding window
        for j in range(1, min(window_size + 1, i + 1)):
            match_length = 0
            # Compare characters from current position with those in the window
            while (i + match_length < len(input_string) and
                   input_string[i + match_length] == input_string[i - j + match_length]):
                match_length += 1
            # Update if we found a longer match
            if match_length > max_match_length:
                max_match_length = match_length
                max_match_index = i - j

        if max_match_length > 0:
            # If there's a match, add tuple (1, offset, length) to output
            compressed_output.append((1, i - max_match_index, max_match_length))
            i += max_match_length
        else:
            # If no match, add tuple (0, current_character) to output
            compressed_output.append((0, input_string[i]))
            i += 1

    return compressed_output

# Set the window size for LZ77 compression
window_size = 58  # Larger window size gives more compression but runs slower
# Compress the input alphabet
compressed_output = lz77_compression(input_alphabet.decode('utf-8'), window_size)

def lz77_write_to_binary_file(compressed_output):
    """
    Write the compressed data to a binary file.
    
    :param compressed_output: List of tuples representing the compressed data
    """
    output_file = 'encoded_lz77.bin'
    with open(output_file, "wb") as file:
        for element in compressed_output:
            if element[0] == 0:
                # If no match, write (0, character)
                file.write(b'\x00')  # No match marker
                file.write(element[1].encode('utf-8'))
            elif element[0] == 1:
                # If there's a match, write (1, offset, length)
                file.write(b'\x01')  # Match marker
                file.write(element[1].to_bytes(2, byteorder='big'))
                file.write(element[2].to_bytes(2, byteorder='big'))

# Write the compressed data to a binary file
lz77_write_to_binary_file(compressed_output)

def lz77_read_from_binary_file():
    """
    Read the compressed data from the binary file.
    
    :return: List of tuples representing the compressed data
    """
    input_file = 'encoded_lz77.bin'
    output = []
    with open(input_file, 'rb') as file:
        while True:
            marker = file.read(1)
            if not marker:
                break  # End of file
            if marker == b'\x00':
                # No match, read character
                character = file.read(1).decode('utf-8')
                output.append((0, character))
            elif marker == b'\x01':
                # Match, read offset and length
                offset = int.from_bytes(file.read(2), byteorder='big')
                length = int.from_bytes(file.read(2), byteorder='big')
                output.append((1, offset, length))
    return output

# Read the compressed data from the binary file
compressed_data = lz77_read_from_binary_file()

def lz77_decompression(compressed_data):
    """
    Decompress the LZ77 compressed data.
    
    :param compressed_data: List of tuples representing the compressed data
    :return: Decompressed string
    """
    decompressed_string = ""
    for item in compressed_data:
        if item[0] == 0:
            # If it's a single character, append it to the result
            decompressed_string += item[1]
        elif item[0] == 1:
            # If it's a match, copy the specified sequence from the already decompressed data
            offset, length = item[1], item[2]
            start = len(decompressed_string) - offset
            for i in range(length):
                decompressed_string += decompressed_string[start + i]
    return decompressed_string

# Decompress the data
decompressed_string = lz77_decompression(compressed_data)

# Define input and output file paths
input_file_lz77 = 'input_lz77.bin'
decoded_file_lz77 = 'decoded_lz77.bin'

# Write the decompressed data to a file
with open(decoded_file_lz77, 'wb') as file:
    file.write(decompressed_string.encode('utf-8'))

# Compare the original and decompressed files
if filecmp.cmp(input_file_lz77, decoded_file_lz77):
    print("\nFiles have the same content.")
else:
    print("\nFiles do not have the same content.")

# Print file size comparison
print(f"Original file size: {os.path.getsize(input_file_lz77)} bytes")
print(f"Compressed file size: {os.path.getsize('encoded_lz77.bin')} bytes")
print(f"Decompressed file size: {os.path.getsize(decoded_file_lz77)} bytes")

### 3.2 LZW (Lempel-Ziv-Welch):

LZW is a universal lossless data compression algorithm created by Abraham Lempel, Jacob Ziv, and Terry Welch. It's widely used in various file formats and compression utilities due to its simplicity and effectiveness.

#### Theory:
- LZW is a dictionary-based algorithm that builds its dictionary dynamically as it processes the input.
- It starts with a dictionary containing all possible single-character strings.
- As it reads input, it adds new, longer strings to the dictionary.
- It replaces strings of characters with single codes.
- The algorithm doesn't need to transmit the dictionary with the compressed data, as it can be reconstructed during decompression.

#### Implementation details:

1. **Encoder Function (`lzw_encoder`)**:
   - Input: The string to be compressed.
   - Output: A list of indices (compressed data) and the dictionary.
   - Process:
     - Initialize the dictionary with single characters.
     - Iterate through the input string:
       - If a sequence is in the dictionary, extend it.
       - If not, output the code for the current sequence, add the new sequence to the dictionary, and reset.

2. **Writing to Binary File**:
   - We write the compressed data to a binary file in two parts:
     - First line: The unique symbols (initial dictionary)
     - Second line: The compressed data (list of indices)

3. **Reading from Binary File**:
   - We read the compressed data from the binary file:
     - First line: Reconstruct the initial dictionary
     - Second line: Get the list of indices for decompression

4. **Decoder Function (`lzw_decoder`)**:
   - Input: Initial symbols and the sequence of indices.
   - Output: The decompressed string.
   - Process:
     - Initialize the dictionary with initial symbols.
     - Iterate through the indices:
       - Translate each index back to its corresponding string.
       - Add new sequences to the dictionary as they're encountered.

5. **File Handling and Verification**:
   - We implement functions to read input files, write compressed data, read compressed data, and decompress.
   - We compare the original and decompressed files to verify correctness.

#### Key Points:
- LZW is particularly effective for text and data with repeated patterns.
- The algorithm adapts to the input data, building an optimal dictionary for the specific content.
- Our implementation uses a simple string-based dictionary for clarity, though more efficient data structures could be used for larger inputs.
- The compression ratio depends on the nature of the input data and the size of the dictionary.

This LZW implementation demonstrates the core principles of the algorithm while providing practical file I/O operations. It's a good balance between simplicity for understanding and functionality for real-world use.

In [ ]:
import os
import filecmp

# Input file path
input_file = 'input_lzw.bin'

# Read the entire content of the input file
with open(input_file, 'rb') as file:
    input_alphabet = file.read()

# List to store unique symbols from the input
unique_symbols = []

def lzw_encoder(input_string):
    """
    Compress the input string using the LZW algorithm.
    
    :param input_string: The string to be compressed
    :return: A tuple containing the compressed output and the dictionary used for compression
    """
    # Populate unique_symbols list
    for x in input_string:
        if x not in unique_symbols:
            unique_symbols.append(x)

    # Create initial dictionary with unique symbols
    dictionary = {}
    for x in input_string:
        if x not in dictionary:
            dictionary[x] = len(dictionary) + 1  # Start from 1, not 0

    compressed_output = []
    current_sequence = input_string[0]
    
    # Main LZW compression loop
    for char in input_string[1:]:
        next_sequence = current_sequence + char
        if next_sequence in dictionary:
            # If sequence exists, continue to next character
            current_sequence = next_sequence
        else:
            # Output code for current sequence
            compressed_output.append(dictionary[current_sequence])
            # Add new sequence to dictionary
            dictionary[next_sequence] = len(dictionary) + 1
            # Reset current sequence
            current_sequence = char

    # Output code for the last sequence
    compressed_output.append(dictionary[current_sequence])
    return compressed_output, dictionary

# Compress the input alphabet
compressed_output, compressed_dictionary = lzw_encoder(input_alphabet.decode('utf-8'))

# Prepare results for file writing
result_for_file = ' '.join(str(number) for number in compressed_output)
symbols_for_file = ' '.join(str(symbol) for symbol in unique_symbols)

# Write compressed data to file
with open('encoded_lzw.bin', 'wb') as f:
    # Write unique symbols on the first line
    f.write(symbols_for_file.encode() + b'\n')
    # Write compressed data on the second line
    f.write(result_for_file.encode())

# Read compressed data from file
with open('encoded_lzw.bin', 'rb') as file:
    first_line_bin = file.readline()  # Read unique symbols
    second_line_bin = file.readline()  # Read compressed data

# Decode binary data to string using utf-8
first_line_string = first_line_bin.decode('utf-8')
second_line_string = second_line_bin.decode('utf-8')

# Split string based on spaces and convert to appropriate types
symbols_LZW = [str(x) for x in first_line_string.split()]  # Unique symbols
indices_LZW = [int(x) for x in second_line_string.split()]  # Compressed data

def lzw_decoder(initial_symbols, index_sequence):
    """
    Decompress the LZW compressed data.
    
    :param initial_symbols: List of initial symbols in the dictionary
    :param index_sequence: List of indices representing the compressed data
    :return: Decompressed string
    """
    # Initialize dictionary with initial symbols
    dictionary = {i + 1: symbol for i, symbol in enumerate(initial_symbols)}
    current_index = len(dictionary) + 1
    result = [dictionary[index_sequence[0]]]

    current_sequence = dictionary[index_sequence[0]]

    # Main LZW decompression loop
    for index in index_sequence[1:]:
        if index in dictionary:
            new_sequence = dictionary[index]
        elif index == current_index:
            # Special case for repeated sequence
            new_sequence = current_sequence + current_sequence[0]
        else:
            raise ValueError('Invalid compressed data')
        
        result.append(new_sequence)

        # Add new sequence to the dictionary
        dictionary[current_index] = current_sequence + new_sequence[0]
        current_index += 1

        # Set current sequence to new sequence
        current_sequence = new_sequence

    return ''.join(result)

# Decompress the data
decompressed_result = lzw_decoder(symbols_LZW, indices_LZW)

print(f"Unique symbols: {unique_symbols}")

# Define input and output file paths
input_file_LZW = 'input_lzw.bin'
decoded_file_LZW = 'decoded_lzw.bin'

# Write the decompressed data to a file
with open(decoded_file_LZW, 'wb') as file:
    file.write(decompressed_result.encode('utf-8'))

# Compare the original and decompressed files
if filecmp.cmp(input_file_LZW, decoded_file_LZW):
    print("\nFiles have the same content.")
else:
    print("\nFiles do not have the same content.")

# Print file size comparison
print(f"Original file size: {os.path.getsize(input_file_LZW)} bytes")
print(f"Compressed file size: {os.path.getsize('encoded_lzw.bin')} bytes")
print(f"Decompressed file size: {os.path.getsize(decoded_file_LZW)} bytes")

### Compresion levels:

In [ ]:
import os  # Import os module for file operations

# Define input and output file paths for each algorithm
input_file_shannon_fano_huffman = 'input_shannon_fano_huffman.bin'
output_file_shannon_fano = 'encoded_shannon_fano.bin'
output_file_huffman = 'encoded_huffman.bin'
output_file_lz77 = 'encoded_lz77.bin'

# Get the size of the input file for Shannon-Fano and Huffman
input_file_size = os.path.getsize(input_file_shannon_fano_huffman)

# Get the size of the Shannon-Fano compressed file
shannon_fano_size = os.path.getsize(output_file_shannon_fano)

# Get the size of the Huffman compressed file
huffman_size = os.path.getsize(output_file_huffman)

# Calculate compression ratio for Shannon-Fano
# Compression ratio = original size / compressed size
compression_ratio_shannon_fano = input_file_size / shannon_fano_size
compression_ratio_shannon_fano = round(compression_ratio_shannon_fano, 3)  # Round to 3 decimal places
print(f"Compression ratio for Shannon-Fano algorithm: {compression_ratio_shannon_fano}")

# Calculate compression ratio for Huffman
compression_ratio_huffman = input_file_size / huffman_size
compression_ratio_huffman = round(compression_ratio_huffman, 3)
print(f"Compression ratio for Huffman algorithm: {compression_ratio_huffman}")

# Define input file path for LZ77
input_file_lz77 = 'input_lz77.bin'
# Get the size of the input file for LZ77
input_file_size_lz77 = os.path.getsize(input_file_lz77)
# Get the size of the LZ77 compressed file
lz77_size = os.path.getsize(output_file_lz77)
# Calculate compression ratio for LZ77
compression_ratio_lz77 = input_file_size_lz77 / lz77_size
compression_ratio_lz77 = round(compression_ratio_lz77, 3)
print(f"Compression ratio for LZ77 algorithm: {compression_ratio_lz77}")

# Define input and output file paths for LZW
input_file_lzw = 'input_lzw.bin'
output_file_lzw = 'encoded_lzw.bin'
# Get the size of the input file for LZW
input_file_size_lzw = os.path.getsize(input_file_lzw)
# Get the size of the LZW compressed file
lzw_size = os.path.getsize(output_file_lzw)
# Calculate compression ratio for LZW
compression_ratio_lzw = input_file_size_lzw / lzw_size
compression_ratio_lzw = round(compression_ratio_lzw, 3)
print(f"Compression ratio for LZW algorithm: {compression_ratio_lzw}")